## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Domino Data Lab Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. You need to make sure that these datasets are uploaded and present in the same directory as this notebook file, since we will be importing these files in Spark and create the following tables under the __`country_club`__ database using Spark SQL.

1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into Spark to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the Domino platform, but for the final deliverable, please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

# Checking Existence of Spark Environment Variables

Make sure your notebook is loaded using a PySpark Workspace. If you open up a regular Jupyter workspace the following variables might not exist

In [2]:
spark

In [3]:
sqlContext

### Run the following if you failed to open a notebook in the PySpark Workspace

This will work assuming you are using Spark in the cloud on domino or you might need to configure with your own spark instance if you are working offline

In [4]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

# Create a utility function to run SQL commands

Instead of typing the same python functions repeatedly, we build a small function where you can just pass your query to get results.

- Remember we are using Spark SQL in PySpark
- We can't run multiple SQL statements in one go (no semi-colon ';' separated SQL statements)
- We can run multi-line SQL queries (but still has to be a single statement)

In [5]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

# Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [6]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
dbs = run_sql('show databases')
print(dbs)
dbs.toPandas()

DataFrame[databaseName: string]


databaseName
0  country_club
1       default

# Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you.

To get started, first make sure you have already uploaded the three CSV files and they are present in the same directory as the notebook.

Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [7]:
# File location and type
file_location_bookings = "./Bookings.csv"
file_location_facilities = "./Facilities.csv"
file_location_members = "./Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true" # this helps Spark to infer schema so as to be able to distinguish between numerical and string data.
# if this is set to false, all data is interpreted and stored as string.
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [8]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



# Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [9]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [10]:
run_sql('use country_club')
run_sql('REFRESH table bookings')
run_sql('REFRESH table facilities')
run_sql('REFRESH table members')
tbls = run_sql('show tables')
tbls.toPandas()

database   tableName  isTemporary
0  country_club    bookings        False
1  country_club  facilities        False
2  country_club     members        False

# Test a sample SQL query

__Note:__ You can use multi-line SQL queries (but still a single statement) as follows

In [11]:
result = run_sql('''
                    SELECT * 
                    FROM bookings 
                    LIMIT 3
                 ''')
result.toPandas()

bookid  facid  memid           starttime  slots
0       0      3      1 2012-07-03 11:00:00      2
1       1      4      1 2012-07-03 08:00:00      2
2       2      6      0 2012-07-03 18:00:00      2

In [12]:
result = run_sql('''
                    SELECT * 
                    FROM facilities 
                    LIMIT 3
                 ''')
result.toPandas()

facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50

In [13]:
result = run_sql('''
                    describe facilities
                 ''')
result.toPandas()

col_name data_type comment
0               facid       int    None
1                name    string    None
2          membercost    double    None
3           guestcost    double    None
4       initialoutlay       int    None
5  monthlymaintenance       int    None

# Your Turn: Solve the following questions with Spark SQL

- Make use of the `run_sql(...)` function as seen in the previous example
- You can write multi-line SQL queries but it has to be a single statement (no use of semi-colons ';')
- Make use of the `toPandas()` function as depicted in the previous example to display the query results

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [14]:
result = run_sql('''
                  select name from facilities where membercost=0.0
                   
                 ''')
result.toPandas()

name
0  Badminton Court
1     Table Tennis
2    Snooker Table
3       Pool Table

####  Q2: How many facilities do not charge a fee to members?

In [15]:
result = run_sql('''select count(*) from facilities where membercost=0.0
                   
                 ''')
result.toPandas()

count(1)
0         4

#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [16]:
result = run_sql('''
select  facid,  name, membercost, monthlymaintenance from facilities where membercost>0.0--and membercost<(.2*monthlymaintenance)
                   
                ''')
result.toPandas()

facid            name  membercost  monthlymaintenance
0      0  Tennis Court 1         5.0                 200
1      1  Tennis Court 2         5.0                 200
2      4  Massage Room 1         9.9                3000
3      5  Massage Room 2         9.9                3000
4      6    Squash Court         3.5                  80

#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [17]:
result = run_sql('''select * from facilities where facid=1
                    union
                   select * from facilities where facid=5
                ''')
result.toPandas()

facid            name  membercost  guestcost  initialoutlay  \
0      1  Tennis Court 2         5.0       25.0           8000   
1      5  Massage Room 2         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                3000

#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [18]:
result = run_sql('''select name,monthlymaintenance,
                    case 
                    when monthlymaintenance>100
                    then 'Expensive'
                    else 'Cheap'
                    end as label
                    from facilities
                 ''')
result.toPandas()

name  monthlymaintenance      label
0   Tennis Court 1                 200  Expensive
1   Tennis Court 2                 200  Expensive
2  Badminton Court                  50      Cheap
3     Table Tennis                  10      Cheap
4   Massage Room 1                3000  Expensive
5   Massage Room 2                3000  Expensive
6     Squash Court                  80      Cheap
7    Snooker Table                  15      Cheap
8       Pool Table                  15      Cheap

#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [19]:
result = run_sql('''select firstname, surname from members where joindate=(select max(joindate) from members)
                    
                 ''')
result.toPandas()

firstname surname
0    Darren   Smith

In [20]:
result = run_sql('''select * from members where joindate=(select max(joindate) from members)
                    
                 ''')
result.toPandas()

memid surname firstname                         address  zipcode  \
0     37   Smith    Darren  3 Funktown, Denzington, Boston    66796   

        telephone recommendedby            joindate  
0  (822) 577-3541          None 2012-09-26 18:08:45

In [21]:
result = run_sql('''select * from facilities --where joindate=(select max(joindate) from members)
                    
                 ''')
result.toPandas()

facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   
5      5   Massage Room 2         9.9       80.0           4000   
6      6     Squash Court         3.5       17.5           5000   
7      7    Snooker Table         0.0        5.0            450   
8      8       Pool Table         0.0        5.0            400   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  
5                3000  
6                  80  
7                  15  
8                  15

####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [25]:
result = run_sql('''  select distinct fac_name, Name from (select f.name as fac_name,m.firstname||' '||m.surname as Name from bookings b,
                                  members m,
                                  facilities f
                                  where b.memid=m.memid and b.facid=f.facid
                                  and f.name like 'Tennis Court%'
                                 ) order by Name, fac_name
                   
                 ''')
result.toPandas()

fac_name               Name
0   Tennis Court 1         Anne Baker
1   Tennis Court 2         Anne Baker
2   Tennis Court 1       Burton Tracy
3   Tennis Court 2       Burton Tracy
4   Tennis Court 1       Charles Owen
5   Tennis Court 2       Charles Owen
6   Tennis Court 2       Darren Smith
7   Tennis Court 1      David Farrell
8   Tennis Court 2      David Farrell
9   Tennis Court 1        David Jones
10  Tennis Court 2        David Jones
11  Tennis Court 1       David Pinker
12  Tennis Court 1      Douglas Jones
13  Tennis Court 1      Erica Crumpet
14  Tennis Court 1     Florence Bader
15  Tennis Court 2     Florence Bader
16  Tennis Court 1        GUEST GUEST
17  Tennis Court 2        GUEST GUEST
18  Tennis Court 1     Gerald Butters
19  Tennis Court 2     Gerald Butters
20  Tennis Court 2   Henrietta Rumney
21  Tennis Court 1         Jack Smith
22  Tennis Court 2         Jack Smith
23  Tennis Court 1    Janice Joplette
24  Tennis Court 2    Janice Joplette
25  Tennis Court 1     Jemima Farrell
26  Tennis Court 2     Jemima Farrell
27  Tennis Court 1        Joan Coplin
28  Tennis Court 1          John Hunt
29  Tennis Court 2          John Hunt
30  Tennis Court 1    Matthew Genting
31  Tennis Court 2  Millicent Purview
32  Tennis Court 1         Nancy Dare
33  Tennis Court 2         Nancy Dare
34  Tennis Court 1    Ponder Stibbons
35  Tennis Court 2    Ponder Stibbons
36  Tennis Court 1   Ramnaresh Sarwin
37  Tennis Court 2   Ramnaresh Sarwin
38  Tennis Court 1         Tim Boothe
39  Tennis Court 2         Tim Boothe
40  Tennis Court 1         Tim Rownam
41  Tennis Court 2         Tim Rownam
42  Tennis Court 1      Timothy Baker
43  Tennis Court 2      Timothy Baker
44  Tennis Court 1        Tracy Smith
45  Tennis Court 2        Tracy Smith

#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [27]:
result = run_sql('''select * from bookings 
limit 3
                   
                 ''')
result.toPandas()

bookid  facid  memid           starttime  slots
0       0      3      1 2012-07-03 11:00:00      2
1       1      4      1 2012-07-03 08:00:00      2
2       2      6      0 2012-07-03 18:00:00      2

In [38]:
result = run_sql('''select * from members  where firstname='GUEST'

                   
                 ''')
result.toPandas()

memid surname firstname address  zipcode       telephone recommendedby  \
0      0   GUEST     GUEST   GUEST        0  (000) 000-0000          None   

    joindate  
0 2012-07-01

In [55]:
result = run_sql(''' select * from ((select f.name as Facility_Name,m.firstname||' '||m.surname as Name ,f.membercost*b.slots as Cost--,b.memid
                                  from bookings b,
                                  members m,
                                  facilities f
                                  where b.memid=m.memid and b.facid=f.facid

                                  and to_date(b.starttime,'yyyy-mm-dd')='2012-09-14'
                                  and f.membercost*b.slots>30
                                  and b.memid<>0
                                  )
                                  
                    union
                    (select f.name as Facility_Name,m.firstname||' '||m.surname as Name,f.guestcost*b.slots as Cost--,b.memid
                                  from bookings b,
                                  facilities f,
                                  members m
                                  where  b.facid=f.facid and m.memid=b.memid

                                  and to_date(b.starttime,'yyyy-mm-dd')='2012-09-14'
                                  and f.guestcost*b.slots>30
                                  and b.memid=0
                                  ))
                                 order by Cost
                   
                 ''')
result.toPandas()

Facility_Name            Name   Cost
0    Squash Court     GUEST GUEST   35.0
1  Massage Room 1  Jemima Farrell   39.6
2    Squash Court     GUEST GUEST   70.0
3  Tennis Court 1     GUEST GUEST   75.0
4  Tennis Court 2     GUEST GUEST   75.0
5  Tennis Court 2     GUEST GUEST  150.0
6  Massage Room 1     GUEST GUEST  160.0
7  Massage Room 2     GUEST GUEST  320.0

#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
result = run_sql('''
                    
                 ''')
result.toPandas()

#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [59]:
result = run_sql(''' select Facility_Name,Sum(Cost) as Revenue from ((select f.name as Facility_Name,f.membercost*b.slots as Cost
                                  from bookings b,
                                  members m,
                                  facilities f
                                  where b.memid=m.memid and b.facid=f.facid
                                  and b.memid<>0
                                  )
                                  
                    union
                    (select f.name as Facility_Name,f.guestcost*b.slots as Cost
                                  from bookings b,
                                  facilities f,
                                  members m
                                  where  b.facid=f.facid and m.memid=b.memid

                                  
                                  and b.memid=0
                                  ))
                                 group by Facility_Name
                                 order by Revenue
                   
                 ''')
result.toPandas()

Facility_Name  Revenue
0     Table Tennis     10.0
1       Pool Table     15.0
2    Snooker Table     30.0
3  Badminton Court    139.5
4   Massage Room 2    499.8
5   Tennis Court 2    540.0
6     Squash Court    637.0
7   Tennis Court 1    840.0
8   Massage Room 1   1659.4